***Necessary Imports***

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import ImageGrid

import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D
from tensorflow.keras.layers import Conv2DTranspose, MaxPooling2D, concatenate, AveragePooling2D, Dense, Flatten, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

***Loading the dataset***

In [ ]:
#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "vrushali25898" # username from the json file
os.environ['KAGGLE_KEY'] = "12a58cd5ea5950100079afa5fef64042" # key from the json file

!kaggle datasets download -d mateuszbuda/lgg-mri-segmentation

In [ ]:
# extracting the zip
import zipfile
with zipfile.ZipFile("/content/lgg-mri-segmentation.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [ ]:
# assigning the datapath
data_path = '/content/data/kaggle_3m/'

# creating a list of files containing the word "mask"
mask_files = glob('/content/data/kaggle_3m/*/*_mask*')

# replacing the mask work by empty string
train_files = [file.replace('_mask', '') for file in mask_files]

In [ ]:
# creating a dataframe for further handling the dataset
df = pd.DataFrame({"image_path": train_files,
                   "mask_path": mask_files})

# diplaying the first 5 rows of dataframe
df.head()
df.count()

In [ ]:
# this is a function defined in order to classify the MRI images into yes or no tumor. If the diagnosis value is 1, the brain has tumor otherwise no
def diagnosis(mask_path):
    value = np.max(cv2.imread(mask_path))
    return 1 if value > 0 else 0

In [ ]:
# the diagnosis column is appended to the dataframe so for us to reconfirm that the function diagnosis works.
df['diagnosis'] = list(map(lambda x: diagnosis(x), mask_files))
df.head()

***Histogram plot of classification results based on diagnosis column***

In [ ]:
ax = df['diagnosis'].value_counts().plot(kind='bar', stacked=True, figsize=(10,6), color=['blue', 'red'])
ax.set_title('Data Distribution')
ax.set_ylabel('Total Images', fontsize=15)
ax.set_xticklabels(['No Tumor', 'Tumor'], fontsize=12, rotation=0)
for i, rows in enumerate(df['diagnosis'].value_counts().values):
    ax.annotate(int(rows), xy=(i, rows+12), ha='center', fontweight='bold', fontsize=15)

***Just to make sure that the dataframe is created properly, plotting few of the images and masks***

In [ ]:
df_positive = df[df['diagnosis']==1].sample(5).values
df_negative = df[df['diagnosis']==0].sample(5).values

def show_data(df, positive=True):
    images = []
    masks = []
    for data in df:
        img = cv2.imread(data[0])
        mask = cv2.imread(data[1])
        images.append(img)
        masks.append(mask)
    images = np.hstack(np.array(images))
    masks = np.hstack(np.array(masks))
    
    fig = plt.figure(figsize=(25,25))
    if positive:
        grid = ImageGrid(fig, 111, nrows_ncols=(3,1), axes_pad=0.5)
    else:
        grid = ImageGrid(fig, 111, nrows_ncols=(2,1), axes_pad=0.5)
    grid[0].imshow(images)
    grid[0].set_title('Images', fontsize=15)
    grid[0].axis('off')
    grid[1].imshow(masks)
    grid[1].set_title('Masks', fontsize=15)
    grid[1].axis('off')
    if positive:
        grid[2].imshow(images)
        grid[2].imshow(masks, alpha=0.4)
        grid[2].set_title('Brain MRI with mask', fontsize=15)
        grid[2].axis('off')
        
show_data(df_positive)
show_data(df_negative, positive=False)

***Pre-Processing the image dataset for us to proceed with U-Net model***

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.10,random_state=42)
df_train, df_val = train_test_split(df_train, test_size=0.20,random_state=42)

print(df_train.values.shape)
print(df_test.values.shape)
print(df_val.values.shape)

***Fixing the constants***

In [ ]:
IMAGE_SIZE = (256, 256)
EPOCHS = 60
BATCH_SIZE = 32
learning_rate = 1e-4

***Data Augmentation and train and validation generator***

In [ ]:
def train_generator(data_frame, batch_size, aug_dict):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        x_col = "image_path",
        class_mode = None,
        color_mode = "rgb",
        target_size = (256,256),
        batch_size = batch_size,
        seed = 2022)

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        x_col = "mask_path",
        class_mode = None,
        color_mode = "grayscale",
        target_size = (256,256),
        batch_size = batch_size,
        seed = 2022)
# yeild because to pick data batch waise
    for (img, mask) in zip(image_generator, mask_generator):
        img, mask = norm_data(img, mask)
        yield (img, mask)

In [ ]:
# when mask value is 0 it indicates that brain has no tumor and viceversa
def norm_data(img, mask):
    img = img / 255.
    mask = mask / 255.
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0 
    return (img, mask)

In [ ]:
train_generator_args = dict(rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest')
train_gen = train_generator(df_train, BATCH_SIZE, train_generator_args)
    
val_gen = train_generator(df_val, BATCH_SIZE, dict())

***Metrics***

**Mathematically dice coefficient is ((2 * |X intersection Y|) / (|X|+|Y|))**

In [ ]:
smoothing_factor = 10
# smoothing factor is added to make sure denominator never goes zero and so the result should not be indefinite
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    return (2.0 * intersection + smoothing_factor) / (union + smoothing_factor)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    return dice_coef_loss(y_true, y_pred) + bce(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + smoothing_factor) / (sum_ - intersection + smoothing_factor)
    return jac

**U-Net without Attention**

In [ ]:
def conv_block(inp,filters):
    x=Conv2D(filters,(3,3),padding='same',activation='relu')(inp)
    x=Conv2D(filters,(3,3),padding='same')(x)
    x=BatchNormalization(axis=3)(x)
    x=Activation('relu')(x)
    return x

def encoder_block(inp,filters):
    x=conv_block(inp,filters)
    p=MaxPooling2D(pool_size=(2,2))(x)
    return x,p

def decoder_block(inp,filters,concat_layer):
    x=Conv2DTranspose(filters,(2,2),strides=(2,2),padding='same')(inp)
    x=concatenate([x,concat_layer])
    x=conv_block(x,filters)
    return x    

In [ ]:
# Define input layer
input_img = Input((IMAGE_SIZE[0], IMAGE_SIZE[1], 3), name='img')

# Define UNet model without attention
d1,p1=encoder_block(input_img,64)
d2,p2=encoder_block(p1,128)
d3,p3=encoder_block(p2,256)
d4,p4=encoder_block(p3,512)
b1=conv_block(p4,1024)
e2_=decoder_block(b1,512,d4)
e3_=decoder_block(e2_,256,d3)
e4_=decoder_block(e3_,128,d2)
e5_=decoder_block(e4_,64,d1)
outputs = Conv2D(1, (1,1),activation="sigmoid")(e5_)
model=Model(inputs=[input_img], outputs=[outputs],name='Unet')

# Plot the model architecture 
plot_model(model, to_file='model_architecture.png', show_shapes=True)


**U-Net with Attention**

In [ ]:
def attention_block(l_layer,h_layer): 
    fg=Conv2D(h_layer.shape[-1],(1,1),padding='same')(l_layer)
    fx=Conv2D(h_layer.shape[-1],(1,1),strides=(2,2),padding='same')(h_layer)
    x=tf.keras.layers.add([fg,fx])
    x=Activation('relu')(x)
    x=Conv2D(1,(1,1),padding='same',activation='sigmoid')(x)
    x=UpSampling2D(size=(2,2))(x)
    x=tf.keras.layers.multiply([h_layer,x])
    x=BatchNormalization(axis=3)(x)
    return x    

def decoder_block_att(inp,filters,concat_layer):
    x=Conv2DTranspose(filters,(2,2),strides=(2,2),padding='same')(inp)
    concat_layer=attention_block(inp,concat_layer)
    x=concatenate([x,concat_layer])
    x=conv_block(x,filters)
    return x

In [ ]:
# Define UNet model with attention
e2=decoder_block_att(b1,512,d4)
e3=decoder_block_att(e2,256,d3)
e4=decoder_block_att(e3,128,d2)
e5=decoder_block_att(e4,64,d1)
outputs_att = Conv2D(1, (1,1),activation="sigmoid")(e5)
model_att=Model(inputs=[input_img], outputs=[outputs_att],name='AttentionUnet')
# Plot the model architecture
plot_model(model_att, to_file='model_architecture_decode_att.png', show_shapes=True)

***Train Model without Attention***

In [ ]:
opt = Adam(learning_rate=learning_rate)
model.compile(optimizer=opt, loss=bce_dice_loss, metrics=[iou, dice_coef])

In [ ]:
best_path = '/content/data/unet_brainMRI.h5'

callbacks = [ModelCheckpoint(best_path, verbose=1, monitor='val_loss', save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=5, verbose=1, min_lr=1e-7),
            EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=10)]

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch=len(df_train) / BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=callbacks,
                    validation_data = val_gen,
                    validation_steps=len(df_val) / BATCH_SIZE)

***Plotting the trends of train vs validation loss, dice_coeff and iou with respect to increase in number of epochs for the unet model without attention***

In [ ]:
plt.figure(figsize=(8,15))
plt.subplot(3,1,1)
plt.plot(model.history.history['loss'], 'b-', label='train_loss')
plt.plot(model.history.history['val_loss'], 'r-', label='val_loss')
plt.legend(loc='best')
plt.title('Loss')

plt.subplot(3,1,2)
plt.plot(model.history.history['iou'], 'b-', label='train_iou')
plt.plot(model.history.history['val_iou'], 'r-', label='val_iou')
plt.legend(loc='best')
plt.title('IoU')

plt.subplot(3,1,3)
plt.plot(model.history.history['dice_coef'], 'b-', label='train_dice_coef')
plt.plot(model.history.history['val_dice_coef'], 'r-', label='val_dice_coef')
plt.legend(loc='best')
plt.title('Dice Coef')

***Loading the best weights of model without attention***

In [ ]:
model.load_weights("/content/data/unet_brainMRI.h5")

***Evaluation of Model without Attention***

In [ ]:
test_gen = train_generator(df_test, BATCH_SIZE, dict())
results = model.evaluate(test_gen, steps=len(df_test) / BATCH_SIZE)
print("Test IOU: ",results[1])
print("Test Dice Coefficent: ",results[2])

In [ ]:
for i in range(10):
    index=np.random.randint(1,len(df_test.index))
    img = cv2.imread(df_test['image_path'].iloc[index])
    img = cv2.resize(img ,IMAGE_SIZE)
    img = img / 255
    img = img[np.newaxis, :, :, :]
    pred=model.predict(img)

    plt.figure(figsize=(12,12))
    plt.subplot(1,3,1)
    plt.imshow(np.squeeze(img))
    plt.title('Original Image')
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(cv2.imread(df_test['mask_path'].iloc[index])))
    plt.title('Original Mask')
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(pred) > .5)
    plt.title('Prediction')
    # Save the figure as an image with a unique filename
    plt.savefig(f'trend_plot_{i}.png')  # Specify a unique filename for each figure
    plt.show()

***Train model with Attention***

In [ ]:
best_path = '/content/data/unet_brainMRI_att.h5'

callbacks = [ModelCheckpoint(best_path, verbose=1, monitor='val_loss', save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=5, verbose=1, min_lr=1e-7),
            EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=10)]

In [ ]:
opt = Adam(learning_rate=learning_rate)
model_att.compile(optimizer=opt, loss=bce_dice_loss, metrics=[iou, dice_coef])

In [ ]:
history = model_att.fit(train_gen,
                    steps_per_epoch=len(df_train) / BATCH_SIZE, 
                    epochs=EPOCHS, 
                    callbacks=callbacks,
                    validation_data = val_gen,
                    validation_steps=len(df_val) / BATCH_SIZE)

***Plotting the trends of train vs validation loss, dice_coeff and iou with respect to increase in number of epochs for the unet model with attention***

In [ ]:
plt.figure(figsize=(8,15))
plt.subplot(3,1,1)
plt.plot(model_att.history.history['loss'], 'b-', label='train_loss')
plt.plot(model_att.history.history['val_loss'], 'r-', label='val_loss')
plt.legend(loc='best')
plt.title('Loss')
# Save the figure as an image file
plt.savefig('loss.png')  # specify the file name and extension

plt.subplot(3,1,2)
plt.plot(model_att.history.history['iou'], 'b-', label='train_iou')
plt.plot(model_att.history.history['val_iou'], 'r-', label='val_iou')
plt.legend(loc='best')
plt.title('IoU')
# Save the figure as an image file
plt.savefig('iou.png')  # specify the file name and extension

plt.subplot(3,1,3)
plt.plot(model_att.history.history['dice_coef'], 'b-', label='train_dice_coef')
plt.plot(model_att.history.history['val_dice_coef'], 'r-', label='val_dice_coef')
plt.legend(loc='best')
plt.title('Dice Coef')
# Save the figure as an image file
plt.savefig('dice_coeff.png')  # specify the file name and extension

***Loading the best weights of model with attention***

In [ ]:
model_att.load_weights("/content/data/unet_brainMRI_att.h5")

***Evaluation of Model with Attention***

In [ ]:
test_gen = train_generator(df_test, BATCH_SIZE, dict())
results = model_att.evaluate(test_gen, steps=len(df_test) / BATCH_SIZE)
print("Test IOU: ",results[1])
print("Test Dice Coefficent: ",results[2])

In [ ]:
for i in range(10):
    index=np.random.randint(1,len(df_test.index))
    img = cv2.imread(df_test['image_path'].iloc[index])
    img = cv2.resize(img ,IMAGE_SIZE)
    img = img / 255
    img = img[np.newaxis, :, :, :]
    pred = model_att.predict(img)

    plt.figure(figsize=(12,12))
    plt.subplot(1,3,1)
    plt.imshow(np.squeeze(img))
    plt.title('Original Image')
    plt.subplot(1,3,2)
    plt.imshow(np.squeeze(cv2.imread(df_test['mask_path'].iloc[index])))
    plt.title('Original Mask')
    plt.subplot(1,3,3)
    plt.imshow(np.squeeze(pred) > .5)
    plt.title('Prediction')
    # Save the figure as an image with a unique filename
    plt.savefig(f'prediction_{i+1}.png')  # Specify a unique filename for each figure
    plt.show()